In [1]:
import os
import sys
#sys.path.append("/workspace")
import numpy as np
import tensorflow as tf
from src.data.general_processor import Utils
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint, EarlyStopping
import pickle
from sklearn.preprocessing import minmax_scale
import pathlib
from sklearn.metrics import classification_report, confusion_matrix


from src.models.model_architectures.model_1DCNN import HopefullNet

%load_ext autoreload
%autoreload 2

In [6]:
root_path = pathlib.Path().resolve().parents[0]
data_path = "data/processed/physionet/2_classes/ours_3_pairs/"
# modelname = "ours_3_pairs/leave_subjects_out/2023-03-31_11-48-20/"
modelname = "ours_3_pairs/leave_subjects_out/test_physio_34_10_65_90_101/2023-04-25_16-09-09_correctedBug/"


## Device settings
tf.autograph.set_verbosity(0)
physical_devices = tf.config.experimental.list_physical_devices('CPU')
print(physical_devices)

#Params
source_path = os.path.join(root_path, data_path)
save_path = os.path.join(root_path, "models/1D_CNN/", modelname)
    
channels = [["C3","C4"],["F3","F4"],["P3","P4"]]

model_path = os.path.join(save_path, "bestModel.h5")
two_class = True

if two_class:
    input_shape = (None, 500, 2)
    loss = tf.keras.losses.binary_crossentropy
else:
    input_shape = (None, 640, 2)
    loss = tf.keras.losses.categorical_crossentropy

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [10]:
# Test subjects for testing transfer learning
test_subjects = [34, 10, 65, 90, 101]

for subj in test_subjects:
    print(f"Test model with subject {subj}:\n")
    subjects = [subj]

    # Load data
    x, y_test = Utils.load(channels, subjects, base_path=source_path)

    # Reshape for scaling
    reshaped_x = x.reshape(x.shape[0], x.shape[1] * x.shape[2])
    reshaped_x_scaled = minmax_scale(reshaped_x, axis=1)
    
#     x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], int(reshaped_x_scaled.shape[1]/2),2).astype(np.float64)
    x_test = reshaped_x_scaled.reshape(reshaped_x_scaled.shape[0], x.shape[1], x.shape[2]).astype(np.float64)
    x_test = np.swapaxes(x_test,1,2)

    # Transform labels in int-values (0/1)
    y_test_01 = []
    for y_label in y_test:
        if y_label == 'L':
            y_test_01.append(0)
        elif y_label == 'R':
            y_test_01.append(1)
        else:
            print("Test Labels are different than L or R...")

    y_test = np.array(y_test_01)
        
    ## Test model with subject data    
    model = HopefullNet(inp_shape = (input_shape[1],input_shape[2]), two_class=two_class)
    model.build(input_shape)
    model.load_weights(model_path)

    learning_rate = 1e-4
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
    
    testLoss, testAcc = model.evaluate(x_test, y_test)
    print('\nAccuracy:', testAcc)
    print('\nLoss: ', testLoss)

    yPred = model.predict(x_test)

    # convert label in string
    if two_class:
        yTestClass = []
        for label in y_test:
            if label==0:
                yTestClass.append("L")
            elif label==1:
                yTestClass.append("R")
            else:
                print("Label not found.")

        yPredClass = []
        for label in yPred:
            if label<0.5:
                yPredClass.append("L")
            elif label>=0.5:
                yPredClass.append("R")
            else:
                print("Label not found.")

        target_names = ["L", "R"]
    else:  
        yTestClass = np.argmax(y_test, axis=1)
        yPredClass = np.argmax(yPred,axis=1)
        target_names=["B", "R", "RL", "L", "F"]

    print('\n Classification report \n\n',
      classification_report(
          yTestClass,
          yPredClass,
           target_names=target_names
      )
    )

    print('\n Confusion matrix \n\n',
      confusion_matrix(
          yTestClass,
          yPredClass,
      )
    )

Test model with subject 34:

Predicting 2 classes.
5/5 [==============================] - 0s 18ms/step - loss: 1.5628 - accuracy: 0.6136

Accuracy: 0.6136363744735718

Loss:  1.562832236289978
5/5 [==============================] - 0s 17ms/step

 Classification report 

               precision    recall  f1-score   support

           L       0.61      0.52      0.56        63
           R       0.62      0.70      0.65        69

    accuracy                           0.61       132
   macro avg       0.61      0.61      0.61       132
weighted avg       0.61      0.61      0.61       132


 Confusion matrix 

 [[33 30]
 [21 48]]
Test model with subject 10:

Predicting 2 classes.
5/5 [==============================] - 0s 19ms/step - loss: 2.2630 - accuracy: 0.5037

Accuracy: 0.5037037134170532

Loss:  2.2630295753479004
5/5 [==============================] - 0s 18ms/step

 Classification report 

               precision    recall  f1-score   support

           L       0.54      0.5